В файле <i>orders</i> представлена информация о заказах в онлайн-магазине.
```
OrderID - ID Заказа,
UserID - ID клиента,
OrderTime - время заказа,
items - список элементов в заказе, каждый элемент описан словарем:
    ItemID  - ID продукта, 
    price - цена единицы товара, 
    quantity - количество единиц товара в заказе
```
В файле <i>categories</i> представлена информация о категории товаров.
```
ItemID - ID продукта,
Category - категория продукта,
ParentCategory - более крупная категория продукта
```

## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.

* Определите, какого числа было сделано больше всего заказов?
* Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)
* Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.

In [1]:
import pandas as pd

In [2]:
categories = pd.read_pickle('data/categories.pkl')
orders = pd.read_pickle('data/orders.pkl')

In [67]:
orders.head()

,OrderID,UserID,OrderTime,items,Date,ItemID,price,quantity,revenue
0,90659865,1.609460e+18,2021-01-01,"{'ItemID': '413736', 'price': 36.72, 'quantity...",2021-01-01,413736,36.72,2,73.44
0,90659865,1.609460e+18,2021-01-01,"{'ItemID': '523171', 'price': 90.32, 'quantity...",2021-01-01,523171,90.32,1,90.32
0,90659865,1.609460e+18,2021-01-01,"{'ItemID': '452831', 'price': 223.11, 'quantit...",2021-01-01,452831,223.11,1,223.11
0,90659865,1.609460e+18,2021-01-01,"{'ItemID': '303673', 'price': 59.9, 'quantity'...",2021-01-01,303673,59.90,7,419.30
0,90659865,1.609460e+18,2021-01-01,"{'ItemID': '318193', 'price': 319.91, 'quantit...",2021-01-01,318193,319.91,2,639.82


In [13]:
categories.head()

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки
2,1000048,Сладости для диабетиков,Сладости
3,1000049,Вода,Напитки
4,1000061,Фрукты,Овощи и фрукты


In [ ]:
# 1.Определите, какого числа было сделано больше всего заказов?

In [14]:
orders['OrderTime'] = pd.to_datetime(orders['OrderTime']).dt.date

In [37]:
order_by_date = orders.groupby('OrderTime').size()
order_by_date.head()

OrderTime
2021-01-01     75
2021-01-02    154
2021-01-03    164
2021-01-04    212
2021-01-05    226
dtype: int64

In [38]:
max_orders = order_by_date.max()
max_orders

np.int64(342)

In [40]:
max_orders_date = order_by_date[order_by_date == max_orders].index[0]

2021-02-05


In [94]:
print(f'Максимальное число заказов произошло {max_orders_date} и составило {max_orders} шт.')

Максимальное число заказов произошло 2021-02-05 и составило 342 шт.


In [ ]:
# 2.Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)

In [60]:
orders = orders.explode('items')

In [63]:
orders['ItemID'] = orders['items'].apply(lambda x: x['ItemID'])
orders['price'] = orders['items'].apply(lambda x: x['price'])
orders['quantity'] = orders['items'].apply(lambda x: x['quantity'])

In [66]:
orders['revenue'] = orders['price'] * orders['quantity']

In [71]:
orders_categories = pd.merge(orders, categories, on = 'ItemID')
orders_categories.head()

,OrderID,UserID,OrderTime,items,Date,ItemID,price,quantity,revenue,Category,ParentCategory
0,90659865,1.609460e+18,2021-01-01,"{'ItemID': '413736', 'price': 36.72, 'quantity...",2021-01-01,413736,36.72,2,73.44,Мороженое,Молочная продукция
1,90659865,1.609460e+18,2021-01-01,"{'ItemID': '523171', 'price': 90.32, 'quantity...",2021-01-01,523171,90.32,1,90.32,Полуфабрикаты,Бакалея
2,90659865,1.609460e+18,2021-01-01,"{'ItemID': '452831', 'price': 223.11, 'quantit...",2021-01-01,452831,223.11,1,223.11,Овощные смеси,Овощи и фрукты
3,90659865,1.609460e+18,2021-01-01,"{'ItemID': '303673', 'price': 59.9, 'quantity'...",2021-01-01,303673,59.90,7,419.30,Фрукты,Овощи и фрукты
4,90659865,1.609460e+18,2021-01-01,"{'ItemID': '318193', 'price': 319.91, 'quantit...",2021-01-01,318193,319.91,2,639.82,Фрукты,Овощи и фрукты


In [77]:
gmv_by_parent_category = orders_categories.groupby('ParentCategory')['revenue'].sum()
gmv_by_parent_category.head()

ParentCategory
Бакалея            25223205.54
Бытовая химия       6690247.78
Для детей           2684441.54
Для дома и сада     1706359.70
Для праздника        406776.16
Name: revenue, dtype: float64

In [80]:
top_3 = gmv_by_parent_category.sort_values(ascending=False).head(3)
top_3

ParentCategory
Мясо, птица, рыба     32268493.46
Молочная продукция    27702995.43
Овощи и фрукты        27607486.90
Name: revenue, dtype: float64

In [ ]:
# Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа.
# Проверьте работу этой функции.

In [89]:
def recommendation(OrderID):
    # Получаем список продуктов и их категории в заказе по OrderID
    items_from_order = orders_categories[orders_categories['OrderID'] == OrderID]['ItemID'].tolist()
    categories_from_order = orders_categories[orders_categories['OrderID'] == OrderID]['Category'].unique()

    # Выбираем из таблицы данные только по категориям, присутствующим в заказе.
    # Затем вычисляем наиболее популярные продукты с помощью группировки и обратной сортировки
    popular_items = orders_categories[orders_categories['Category'].isin(categories_from_order)]
    popular_items = popular_items.groupby('ItemID').size().sort_values(ascending=False)

    # Перебираем популярные товары и проверяем, есть ли они в заказе
    # Если нет, функция возвращает ItemID (предлагает добавить товар в корзину)
    for item in popular_items.index:
        if item not in items_from_order:
            return item

In [93]:
OrderID = 90659865
rec_item = recommendation(OrderID)
print(f'Для заказа № {OrderID} рекомендуется приобрести товар № {rec_item}')

Для заказа № 90659865 рекомендуется приобрести товар № 430560
